# Intelligent Driver Model Notebook (Done)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from data.param_data_loader2 import ParamDL2
dataset = ParamDL2()

dataset.summarize()

Datast Loaded. Preview of dataset: 
   driver  trip  congestion  cf_idx  type    time   speed  distance  \
0       1    53           0       0     0  1698.8  6.3472   33011.0   
1       1    53           0       0     0  1698.9  6.3194   33012.0   
2       1    53           0       0     0  1699.0  6.3083   33013.0   
3       1    53           0       0     0  1699.1  6.2556   33013.0   
4       1    53           0       0     0  1699.2  6.2389   33014.0   

   accelpedal  brakepedal  ...  timeHeadway  VL_theta  VL_dtheta  lv_speed  \
0         0.0           1  ...     2.294240  0.137128   0.009974    5.2847   
1         0.0           1  ...     2.304333  0.137128   0.009974    5.2569   
2         0.0           1  ...     2.278744  0.138907   0.009632    5.3083   
3         0.0           1  ...     2.277959  0.140121   0.009801    5.2556   
4         0.0           1  ...     2.264021  0.141357   0.010598    5.1764   

     lv_Ax  distanceCalibrated   lv_distance  timeSinceCF  distanceS

In [3]:
# Import models, before other cells, so there is no need to rerun some tests
from param_models import *
model = BatchOVIDM(dataset, seqs=["jam"], mode='gpu')

In [ ]:
import time

# Test using RMSE over Gap
objective = ["s_ego_pred_rmse", "relv_ego_pred_rmse"]

# Simulate and verify that it works
def_params = model.hyperparams.from_default(device=model.device)

# Simulate
t_s = time.perf_counter()
model.simulate(def_params, batch_size=1, pbar=True)
print(f"{model.model_name} simulated in", time.perf_counter()-t_s, "seconds")

# Compute Stats
t_s = time.perf_counter()
stats = model.compute_stats(objective)
print(f"{model.model_name} stats computed in", time.perf_counter()-t_s, "seconds")
if isinstance(objective, list):
    for obj, stat in zip(objective, stats):
        print("Baseline parameters produced", obj, "=", stat["jam_"+obj+"_min"])

print(f"Graphing {model.model_name}")
seqs = model.get_seqs(num_samples=0,groups=["jam"], method="sequential")
model.plot_sim_results(seqs,out_dir="results/"+model.model_name+"_default", label="default", show=False, debug=False)

100%|██████████| 6278/6278 [00:04<00:00, 1310.27it/s]


OVIDM simulated in 4.792984763975255 seconds
OVIDM stats computed in 0.007861114980187267 seconds
Baseline parameters produced s_ego_pred_rmse = tensor(4.5037, device='cuda:0')
Baseline parameters produced relv_ego_pred_rmse = tensor(0.6227, device='cuda:0')
Graphing OVIDM


 13%|█▎        | 8/60 [00:03<00:23,  2.23it/s]


KeyboardInterrupt: 

In [ ]:
RETUNE = False

best_params = [ 1.06452546, 0.5, 4.29951906, 2.05361199, 39.99809795, 0.01, 12.2, 0.389]  # From a previous tune cycle
objective = "s_ego_pred_rmsne"

if RETUNE == True:
    best_params = model.ga_calibrate(objective, "jam", sol_size=1024, batch_size=256, num_generations=500)

opt_params = model.hyperparams.from_array(best_params, model.device)
model.simulate(opt_params, batch_size=1)
stats = model.compute_stats(objective)

seqs = model.get_seqs(num_samples=0,groups=["jam"], method="sequential")
model.plot_sim_results(seqs, out_dir="results/"+model.model_name+"_overall_optimized", label="overall_best", show=False, debug=False)

In [ ]:
objective = "s_ego_pred_rmsne"
opt_dict = model.per_param_optimize(best_params, objective)
model.plot_per_param_opt_overview(opt_dict)

In [ ]:
model.plot_per_param_opt_indv(opt_dict, best_idx=3)